In [1]:
%cd /content/drive/MyDrive/대구폴/화/딥러닝 수학/pytorch
!pwd

/content/drive/MyDrive/대구폴/화/딥러닝 수학/pytorch


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

from file_read import read_value

In [4]:
final_points, cnt = read_value("score_mlr03.txt")

In [5]:
cnt

25

In [8]:
final_points

[[73.0, 80.0, 75.0, 152.0],
 [93.0, 88.0, 93.0, 185.0],
 [89.0, 91.0, 90.0, 180.0],
 [96.0, 98.0, 100.0, 196.0],
 [73.0, 66.0, 70.0, 142.0],
 [53.0, 46.0, 55.0, 101.0],
 [69.0, 74.0, 77.0, 149.0],
 [47.0, 56.0, 60.0, 115.0],
 [87.0, 79.0, 90.0, 175.0],
 [79.0, 70.0, 88.0, 164.0],
 [69.0, 70.0, 73.0, 141.0],
 [70.0, 65.0, 74.0, 141.0],
 [93.0, 95.0, 91.0, 184.0],
 [79.0, 80.0, 73.0, 152.0],
 [70.0, 73.0, 78.0, 148.0],
 [93.0, 89.0, 96.0, 192.0],
 [78.0, 75.0, 68.0, 147.0],
 [81.0, 90.0, 93.0, 183.0],
 [88.0, 92.0, 86.0, 177.0],
 [78.0, 83.0, 77.0, 159.0],
 [82.0, 86.0, 90.0, 177.0],
 [86.0, 82.0, 89.0, 175.0],
 [78.0, 83.0, 85.0, 175.0],
 [76.0, 83.0, 71.0, 149.0],
 [96.0, 93.0, 95.0, 192.0]]

## weight, bias의 초기값은 랜덤 값으로 셋팅됨
- model().parameter -> random값이 들어가있음
- iteration or epoch 돌면서 model parameter의 값들을 업데이트
- 실행마다 결과값이 다른 것을 방지하기 위해
```torch.manual_seed(1)``` 사용 가능

In [7]:
torch.manual_seed(1)

train data를 일단 가져오고, train data로부터 x,y 값을 구분해야 함
- x = [mid, final, report]
- y = [final points]

In [10]:
train_data = np.array(final_points)
train_data.shape

(25, 4)

In [11]:
x_train = train_data[:-5, :-1]  # 20x3 생성
y_train = train_data[:-5, -1]

In [12]:
x_train

array([[ 73.,  80.,  75.],
       [ 93.,  88.,  93.],
       [ 89.,  91.,  90.],
       [ 96.,  98., 100.],
       [ 73.,  66.,  70.],
       [ 53.,  46.,  55.],
       [ 69.,  74.,  77.],
       [ 47.,  56.,  60.],
       [ 87.,  79.,  90.],
       [ 79.,  70.,  88.],
       [ 69.,  70.,  73.],
       [ 70.,  65.,  74.],
       [ 93.,  95.,  91.],
       [ 79.,  80.,  73.],
       [ 70.,  73.,  78.],
       [ 93.,  89.,  96.],
       [ 78.,  75.,  68.],
       [ 81.,  90.,  93.],
       [ 88.,  92.,  86.],
       [ 78.,  83.,  77.]])

In [13]:
y_train

array([152., 185., 180., 196., 142., 101., 149., 115., 175., 164., 141.,
       141., 184., 152., 148., 192., 147., 183., 177., 159.])

# pytorch 입력은 Tensor이므로 바꿔주기

In [14]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

# 모델 선언, 가중치 초기화
```nn.Linear(input dimension, output dimension)```

In [46]:
model = nn.Linear(3, 1)

# optimizer setting
optimizer = optim.SGD(model.parameters(), lr=0.000005)  

# training

In [47]:
nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    # forwarding ( H(x) 계산 : wx + b )
    pred = model(x_train)  # predict

    cost = F.mse_loss(pred, y_train)  # cost가 최소화 되도록 학습

    optimizer.zero_grad()  # 누적 방지

    cost.backward()  # backward 연산
    optimizer.step()  # weight, bias update

    if epoch % 100 == 0:
        print(f"Epoch : {epoch:4d}, Cost : {cost.item():.4f}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  import sys


Epoch :    0, Cost : 49952.4688
Epoch :  100, Cost : 1214.0933
Epoch :  200, Cost : 1211.5903
Epoch :  300, Cost : 1209.1920
Epoch :  400, Cost : 1206.8939
Epoch :  500, Cost : 1204.6917
Epoch :  600, Cost : 1202.5811
Epoch :  700, Cost : 1200.5585
Epoch :  800, Cost : 1198.6196
Epoch :  900, Cost : 1196.7616
Epoch : 1000, Cost : 1194.9808


In [48]:
model_params = list(model.parameters())
model_params[0], model_params[1]

(Parameter containing:
 tensor([[0.6537, 0.6778, 0.6457]], requires_grad=True), Parameter containing:
 tensor([-0.2265], requires_grad=True))

# test  dataset 가져오기

In [49]:
# train_data
x_test = np.array(final_points)
x_test = x_test[-5:, :-1]

x_test = torch.FloatTensor(x_test)

In [50]:
pred_y = model(x_test)
pred_y

tensor([[169.7824],
        [169.0401],
        [161.9056],
        [151.5580],
        [186.9072]], grad_fn=<AddmmBackward0>)